In [99]:
import pandas as pd
import numpy as np

import re
import jellyfish

# visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': small,
          'figure.figsize': (14, 9),
          'axes.labelsize': small,
          'axes.titlesize': small,
          'xtick.labelsize': small,
          'ytick.labelsize': small,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

%matplotlib inline

In [2]:
# Загружаем БД:
df = pd.read_excel('data/data.xlsx')

In [3]:
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

df = df[df['shop_name'].isin(target_shops)].copy()
df['shop_name'] = df['shop_name'].apply(lambda x: shops_rename[x])
df.sample()

,shop_name,name,quantity,price,sum
2365,ДИКСИ,ПЕРЕЦ ЧИЛИ ОСТРЫЙ КОТАНИ МЕЛЬН,1.0,119.9,119.9


In [4]:
# Выбираем только магазин ПЯТЕРОЧКА:
df = df[df['shop_name'] == 'ПЯТЕРОЧКА'].drop_duplicates('name').reset_index(drop=True)
df.head()

,shop_name,name,quantity,price,sum
0,ПЯТЕРОЧКА,*3074007 Морковь мытая 1кг,0.71,55.99,39.75
1,ПЯТЕРОЧКА,3248992 КРАСНАЯ ЦЕНА Огурцы марин.720мл,1.00,63.59,63.59
2,ПЯТЕРОЧКА,*19134 GR.R.Горошек зеленый ж/б 425мл,1.00,54.99,54.99
3,ПЯТЕРОЧКА,3334702 Ч.ЛИН.Морож.плом.ван.12% 450г,1.00,339.99,339.99
4,ПЯТЕРОЧКА,*3696849 ПР.НАСТР.Прян.с шок/ап/лим.230г,1.00,59.99,59.99


In [5]:
# Смотрим, в каком виде существуют данные:
print('\n'.join(df['name'].head(20).values))

*3074007 Морковь мытая 1кг
3248992 КРАСНАЯ ЦЕНА Огурцы марин.720мл
*19134 GR.R.Горошек зеленый ж/б  425мл
3334702 Ч.ЛИН.Морож.плом.ван.12% 450г
*3696849 ПР.НАСТР.Прян.с шок/ап/лим.230г
3413519 БР-ЛИТ.Масло сл/сл.в/с82,5%180г
*3463717 HAAS Ароматиз.ВАНИЛИН пищ.1,5г
*4038313 DR.TR.Игр-карт.Б.ГАСТ.д/н.игры
*3371481 Нап.пив.KRONENB.1664БЛ.0.46л
*3141246 Напиток Черноголовка Байкал 2л
3437426 Грунт универсальный 5л
3300573 Пакет ПЯТЕРОЧКА 65х40см
*2157107 COLG.З/п МАКС БЛЕСК 100мл
3380551 IN.Сол.МОРС.пищ.м/м мел/п 0,5кг
*88780 KOTANYI Перец черный мол,пак20г
2250692 Дск Огурцы короткоплодные 450г
99593 ТРИ КОРОЧ.Сух.рж.холод/хрен 40г
*3385945 KOTANYIПриправаИТАЛ.ТРАВЫпак14г
*88781 KOTANYI Чеснок изм.пак 28г
*3409453 СИГАРЕТЫ BOND ST.COMP.BLUE


In [6]:
# Избавляемся от цифр в начале:
df['name'] = df['name'].apply(lambda x: ' '.join(x.split()[1:]))
df.head()

,shop_name,name,quantity,price,sum
0,ПЯТЕРОЧКА,Морковь мытая 1кг,0.71,55.99,39.75
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин.720мл,1.00,63.59,63.59
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б 425мл,1.00,54.99,54.99
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12% 450г,1.00,339.99,339.99
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим.230г,1.00,59.99,59.99


#### Работаем с весом, объёмом товара и количества:

In [7]:
# Находим вес или объём товара:

def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''
    

df['size'] = df['name'].apply(find_weight_or_volume)
df.head()

,shop_name,name,quantity,price,sum,size
0,ПЯТЕРОЧКА,Морковь мытая 1кг,0.71,55.99,39.75,1кг
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин.720мл,1.00,63.59,63.59,720мл
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б 425мл,1.00,54.99,54.99,425мл
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12% 450г,1.00,339.99,339.99,450г
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим.230г,1.00,59.99,59.99,230г


In [8]:
# Смотрим, какие записи не получилось распознать:
print('\n'.join(df[df['size'] == '']['name'].values))

DR.TR.Игр-карт.Б.ГАСТ.д/н.игры
СИГАРЕТЫ BOND ST.COMP.BLUE
САДОК Сем.Пет.Хор.нас.F1 см.окр
СИГАРЕТЫ LD AUT.IMP.COMP.
Зелень свежая в ассортименте
СИГАРЕТЫ WINSTON BLUE
РОСК.РОС.Наб.пасх.ЗАБАВ.ЖИВОТН.
СИГАРЕТЫ SOBRANIE СИНИЕ
СИГАРЕТЫ WINSTON WHITE
АРТ И ДИЗАЙ.Сумка EXLARGE плас.
РОСК.РОС.Наб.пасх.ВЕСЕЛ.ЖИТЕЛИ
СИГАРЕТЫ PHILIP MOR.COMP.BL.
Напиток миндальный обогащ кальц
ECON.Рожок д/обуви
СИГАРЕТЫ MARLBORO COMPACT
Напиток кокосовый с рисом обог
САДОК Семена Шпинат Матадор
САДОК Сем.Горох ов.Детский сах.
САДОК Сем.Горох овощ.Сах.принц
СИГАРЕТЫ ЯВА ЗОЛОТАЯ ОРИГИН.
СИГАРЕТЫ LD AUT.CLUB PLAT.
РОСК.РОС.Наб.пасх.ПУШИСТИКИ
РОСК.РОС.Наб.пасх.ПУШИСТ.УТЯТА
СИГАРЕТЫ PARLIAMENT SILVER BLUE
Пельмени Останкинские Традицион
СИГАРЕТЫ ДОНСКОЙ ТАБАК СВ.
ВОР.Сух.ГРИБЫ СО СМЕТАНЕ рж-пш
ECONTA Фольга алюминиевая бытов
Газета КОМСОМОЛЬСК.ПРАВДА ежен
DIROL Жев.рез.WHITE Клуб.поляна
СИГАРЕТЫ LD AUTOGRAPH RED
COLG.З/щ МАССАЖЕР ср/ж
Кн.Футбол с наклейками
СИГАРЕТЫ KENT БЛЮ
РОСК.РОС.Наб.пасх.ВЕСЕН.БАБОЧКИ
СИГАРЕТЫ 

##### Вроде бы получилось распознать почти всё

In [9]:
def remove_size_from_name(name: str, size: str)-> str:
    '''Удаляет из названия товара цену.'''

    return name.replace(size, '').strip().strip('.')

df[['name']] = df[['name', 'size']].apply(lambda x: remove_size_from_name(**x), axis=1)
df.head()

,shop_name,name,quantity,price,sum,size
0,ПЯТЕРОЧКА,Морковь мытая,0.71,55.99,39.75,1кг
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин,1.00,63.59,63.59,720мл
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б,1.00,54.99,54.99,425мл
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12%,1.00,339.99,339.99,450г
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим,1.00,59.99,59.99,230г


### Находим производителя:

In [256]:
brands = pd.read_csv('data/brands.csv')['0']
print(brands.shape)
brands.head()

(10523,)


0               тчн!
1      играем вместе
2       home element
3             silver
4    птичка-весничка
Name: 0, dtype: object

In [255]:
brands[brands != 'хозяйство васильевой а.в.'].to_csv('data/brands.csv', index=False)
brands = pd.read_csv('data/brands.csv')['0']

In [284]:
similar_brands = brands[brands.apply(lambda x: x.startswith('DR KORN'.lower()))].values
similar_brands

array([], dtype=object)

In [264]:
similar_brands[np.argmax([jellyfish.jaro_distance('LD', x) for x in similar_brands])]

'любятово'

In [270]:
black_list = [
    'ВАНИЛИН', 'СИГАРЕТЫ', 'НАРЕЗНОЙ', 'МАНГО', 'АПЕЛ', 'МОЛОЧНЫЕ', 'МОЛОЧНЫЕ', 'КУКР', 'ST', 'COMP', 'BLUE',
    'ДОМ', 'ХЛОП', 'МАРИЯ', 'ТРАДИЦИОННОЕ', 'ULTRA', 'ЛЕС', 'ЯГ', 'МЕДВЕЖ', 'КОЛЬЦА', 'ВАН', 'СТР', 'РЕЦ',
    'ВЫГОДНО', 'СЛИВОЧНЫЙ', 'ГЕРКУЛЕС', 'ULTRA', 'SUPER', 'ИТАЛ', 'ГРЕЧЕСК', 'БЗМЖ', 'C', 'L', 'Б', 'С',
    'КРАБ', 'ДЕРЕВЕНСК', 'C1', 'MINI', 'КРАБОВЫЙ', 'КАРБОНАРА', 'СОСИСКА', 'РУ', 'ГРЕЧЕСК', 'ХЛЕБ',
    'С', 'ОТРУБЯМИ','АРМЯНСКИЙ',  'ИЗЮМ', 'КИШМИШ', 'ВЕНГЕРСКАЯ', 'ПОСЫПКА', 'ЦВ', 'ШАР', 'СВЕТ',
    'ЛИМ', 'ВКУС', 'ВОЗДУШНЫЙ', 'ПЭТ', 'ГРАНАТОВЫЙ', 'ЩЕЛКОВСКИЙ'
]

def find_brand(name: str)-> str:
    """Находит бренд из слов в верхнем регистре."""
    
    pattern = r'(\b[А-Я]+[0-9]*\b|\b[A-Z]+[0-9]*\b)' # Ищет слова в верхнем регистре
    result = re.findall(pattern, name)
    
    [result.remove(x) for x in result if x in black_list] # удаляю мусор из найденных брендов
    result = [x.lower() for x in result]
    if not result:
        return 'None'
    result_str = ' '.join(result)
    similar_brands = brands[brands.apply(lambda x: x.startswith(result[0].lower()))].values
    dist = [jellyfish.jaro_distance(result_str, x) for x in similar_brands]
    if not dist:
        return 'None'
            
    return similar_brands[np.argmax(dist)]

In [268]:
df['brand'] = df['name'].apply(find_brand)

In [277]:
def remove_brand_from_name(name: str, brand: str)-> str:
    """Удаляет бренды, которые получилось найти из названия товара."""

    if brand == 'None':
        return name.replace('.', ' ').lower()

    result = []
    for word in name.replace('.', ' ').lower().split():
        if brand.startswith(word):
            continue
        result.append(word)
    return ' '.join(result)

In [329]:
df['clear_name'] = df[['name', 'brand']].apply(lambda x: remove_brand_from_name(x['name'], x['brand']), axis=1)

### Работаем с наименованием продука

In [431]:
product_dict = { # 2518 слов можжно заменить этим словарём
    'марин': 'маринованные',
    'морож': "мороженное",
    'плом': 'пломбир',
    'ван': 'ванильный',
    'сл': 'сливочное(слоеное)',
    'ароматиз': 'ароматизатор',
    'пищ': 'пищевой',
    'нап': 'напиток',
    'напи': 'напиток',
    'напит': 'напиток',
    'пив': 'пиво',
    'сух': 'сухой(сухарь)',
    'изм': 'измельчённый',
    'пельм': 'пельмени',
    'завтр': 'завтрак',
    'молшок': 'молочный шоколад',
    'бискв': 'бисквит',
    'глад': 'гладкие',
    'хлоп': 'хлопья',
    'конф': 'конфеты',
    'шокол': 'шоколад',
    'слив': 'сливочное',
    'сгущ': 'сгущеное',
    'конд': 'кондитерская',
    'слад': 'сладкая',
    'лес': 'лесные',
    'яг': 'ягоды',
    'медвеж': 'медвежье',
    'эрив': 'эривань',
    'национал': 'национальный',
    'карт': 'картотфель',
    'хруст': 'хрустящий',
    'мин': 'минеральная',
    'пит': 'питьевая',
    'мор': 'морская',
    'среднепл': 'среднеплодные',
    'майон': 'майонез',
    'мак': 'макароны',
    'кур': 'курица',
    'лап': 'лапша',
    'печ': 'печенье',
    'салф': 'салфетки',
    'нарез': 'нарезной',
    'бум': 'бумага',
    'бел': 'белый(ая)',
    'прян': 'пряники',
    'классич': 'классический',
    'пломб': 'пломбир',
    'обезж': 'обезжиренный',
    'итал': 'итальянский',
    'свет': 'светлое',
    'шок': 'шоколад(ный)',
    'московск': 'московский(ая)',
    'проклад': 'прокладка',
    'кукур': 'кукуруза(ой)',
    'бумаж': 'бумажое',
    'деревенск': 'деревенское(ая)',
    'клуб': 'клубника',
    'абр': 'абрикос',
    'виш': 'вишня(ей)',
    'охл': 'охлаждённое(ые)',
    'бзмж': 'без заменителей молочного жира',
    'йог': 'йогурт',
    'карам': 'карамель(ю)',
    'черн': 'черника(черникой)',
    'соев': 'соевый',
    'газ': 'газированный',
    'туал': 'туалетный(ая)',
    'зер': 'зерновой(ая)',
    'зерн': 'зерновой(ая)',
    'слоен': 'слоеное',
    'ябл': 'яблоко',
    'перс': 'персик',
    'груд': 'грудка',
    'охл': 'охлаждённая',
    'тон': 'тонкий(ая)',
    'упаков': 'упаковка(е)',
    'цв': 'цветной(ых)',
    'подл': 'подложка(е)',
    'батонч': 'батончик',
    'бат': 'батон(чик)',
    'шоколадн': 'шоколадный',
    'глаз': 'глазурь',
    'апел': 'апельсин',
    'теляч': 'телятина',
    'древ': 'древесный',
    'туал': 'туалет',
    'подс': 'подсолнечное',
    'кур': 'куриный(ая)',
    'свин': 'свинина',
    'круас': 'круасаны',
    'дет': 'детский',
    'базил': 'базилик',
    'овс': 'овсяные',
    'пос': 'посуда',
    'нат': 'натуральный(ая)',
    'обжар': 'обжаренные',
    'трюф': 'трюфель',
    'элит': 'элитный',
    'проклад': 'прокладка',
    'майон': 'майонез',
    'сгущ': 'сгущёнка',
    'вар': 'вареная(ый)',
    'готов': 'готовый(ая)',
    'жев': 'жевательная',
    'рез': 'резинка',
    'рж': 'ржаная(ой)',
    'лик': 'ликер',
    'бифшт': 'бифштекс',
    'сосис': 'сосиски',
    'твор': 'творог(твороженный)',
    'ассорт': 'ассортимент(е)',
    'бискв': 'бисквит',
    'биск': 'бисквит',
    'лаком': 'лакомство',
    'класс': 'классическое(ий)',
    'прем' : 'премиум',
    'пшенич' : 'пшеничный(ая)',
    'кваш' : 'квашенный(ая)',
    'кетч' : 'кетчуп',
    'пирож': 'пироженное',
    'подуш': 'подушечки',
    'нач': 'начинка',
    'изд': 'изделие(я)',
    'пюр': 'пюре',
    'темн': 'темный',
    'колб': 'колбаса',
    'наб': 'набор',
}

In [319]:
df.head(10)

,shop_name,name,quantity,price,sum,size,brand,clear_name
0,ПЯТЕРОЧКА,Морковь мытая,0.71,55.99,39.75,1кг,None,морковь мытая
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин,1.00,63.59,63.59,720мл,красная цена,цена огурцы марин
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б,1.00,54.99,54.99,425мл,green ray,r горошек зеленый ж/б
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12%,1.00,339.99,339.99,450г,чистая линия,лин морож плом ван 12%
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим,1.00,59.99,59.99,230г,пряничное настроение,настр с шок/ап/лим
5,ПЯТЕРОЧКА,"БР-ЛИТ.Масло сл/сл.в/с82,5%",1.00,139.99,139.99,180г,бриз,"бр-лит масло сл/сл в/с82,5%"
6,ПЯТЕРОЧКА,HAAS Ароматиз.ВАНИЛИН пищ,4.00,3.74,14.96,"1,5г",haas,ароматиз ванилин пищ
7,ПЯТЕРОЧКА,DR.TR.Игр-карт.Б.ГАСТ.д/н.игры,1.00,0.00,0.00,,dry ru,tr игр-карт б гаст д/н игры
8,ПЯТЕРОЧКА,Нап.пив.KRONENB.1664БЛ,6.00,66.99,401.94,0.46л,kronenbourg,нап пив 1664бл
9,ПЯТЕРОЧКА,Напиток Черноголовка Байкал,1.00,69.99,69.99,2л,None,напиток черноголовка байкал


In [330]:
def remove_one_char(name: str) -> str:
    """Удаляет отдельно стояющую букву."""

    name = name.split()
    words = []
    for word in name:
        if len(word) == 1 and word not in ['в', 'c']:
            continue
        words.append(word)
    return ' '.join(words)

In [331]:
df['clear_name'] = df['clear_name'].apply(remove_one_char)

In [438]:
def remove_two_chars(name: str)-> str:
    name = name.split()
    words = []
    for word in name:
        if len(word) == 2 and '%' not in word and word not in ['из']: 
            continue
        words.append(word)
    return ' '.join(words)

In [439]:
df['clear_name'] = df['clear_name'].apply(remove_two_chars)

In [440]:
def use_product_dict(name):
    words = []
    name = name.split()
    for word in name:
        words.append(product_dict.get(word, word))
    return ' '.join(words)

In [442]:
df['clear_name'] = df['clear_name'].apply(use_product_dict)

## TODO

1) Удалить анлийские слова длиной 3

2) Использовать pymorf

In [446]:
df[['name', 'brand', 'clear_name']].sample(20)

,name,brand,clear_name
750,РАЭ Кисель вишнев.бут,None,раэ кисель вишнев бут
2456,СЛАВ.Конф.ЗОЛОТОЙ СТЕП гл,слава,конфеты золотой степ
1689,MAKFA Мука в/с фас,makfa,мука в/с фас
3162,НАТ.Салат ВИНЕГРЕТ,натурино,салат винегрет
1901,Яблоки Богатырь,None,яблоки богатырь
3379,С.ПР.Сок М/ОВОЩ.С/СВЕКЛ.д/пит,приморские,сок м/овощ с/свекл д/пит
1001,К.Ц.Масло подсолн.раф,куриное царство,масло подсолн раф
2755,"АГУША Нап.БИОЛАКТ к/м 3,2%",агуша,"напиток биолакт к/м 3,2%"
4216,ВОЛОГЖ.Сырок твор/изюм.9%,вологжанка,сырок твор/изюм 9%
1159,Сок ДОБРЫЙ томатный,добрый,сок томатный
